In [195]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision
!apt-get install -y make

import os
sess = boto3.Session()
account_id = sess.client('sts').get_caller_identity()["Account"]
os.environ['DATASETS_BUCKET'] = f'sagemaker-test-mnist-{sess.region_name}-{account_id}'
os.environ['DATASETS_KEY_PREFIX'] = 'mnist'
!make upload_datasets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.2.1-1.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
python download_datasets.py
python upload_datasets.py


In [ ]:
from sagemaker.pytorch import PyTorch
import sagemaker

estimator = PyTorch(
    source_dir='/root/sagemaker-pytorch-mnist',
    entry_point='training.py',
    output_path=f's3://{os.environ["DATASETS_BUCKET"]}/artifacts',
    role=sagemaker.get_execution_role(),
    framework_version='1.3.1',
    py_version='py3',
    train_instance_count=2,
    train_instance_type='ml.g4dn.xlarge',
    hyperparameters={
        'epochs': 10,
        'backend': 'gloo',
        'dropout': 0.2,
    },
    metric_definitions=[
        {'Name':'test:accuracy', 'Regex':'test accuracy: (.*?),'},
        {'Name':'test:loss', 'Regex':'test loss: (.*?);'}
    ],
    enable_sagemaker_metrics=True,
)

inputs = sagemaker.inputs.s3_input(f's3://{os.environ["DATASETS_BUCKET"]}/{os.environ["DATASETS_KEY_PREFIX"]}')

estimator.fit(inputs={'training': inputs})

In [204]:
from sagemaker.pytorch.model import PyTorchModel

training_job_name = 'pytorch-training-2020-07-25-08-41-45-674'
training_job = sess.client('sagemaker').describe_training_job(TrainingJobName=training_job_name)
model = PyTorchModel(model_data=training_job['ModelArtifacts']['S3ModelArtifacts'], 
                     role=sagemaker.get_execution_role(),
                     framework_version='1.3.1',
                     py_version='py3',
                     source_dir='/root/sagemaker-pytorch-mnist',
                     entry_point='inference.py')

model.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1, endpoint_name='pytorch-mnist-test', accelerator_type='ml.eia2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-----------------!

In [206]:
from torchvision import datasets, transforms

dataset = datasets.MNIST('mnist', train=False, transform=transforms.ToTensor(), download=False)
result = predictor.predict(dataset[0][0].view(-1, 1, 28, 28))
print(result) # [[7]]
print(dataset[0][1]) # 7

[[7]]
7


In [207]:
predictor.delete_endpoint()
predictor.delete_model()